<a href="https://colab.research.google.com/github/ManeeshGuptaNvss/ML_Projects/blob/main/NLP_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tokenization

In [189]:
import re

def tokenize(text):

    text = text.lower() 

    # decontraction 
    # regex_dict ={
    #     "won\'t":"will not",
    #     "can\'t": "can not",
    #     "n\'t": " not",
    #     "\'re": " are", 
    #     "\'s": " is", 
    #     "\'d": " would", 
    #     "\'ll": " will", 
    #     "\'t": " not", 
    #     "\'ve": " have", 
    #     "\'m": " am" 
    # }
    
    # for key,value in regex_dict.items():
    #   text = re.sub(key,value,text)
    
    #  removing all commas in huge numbers
    regex_to_modify_numbers = '(?<=\d),(?=\d)'

    pattern1 = re.compile(regex_to_modify_numbers)
    text = re.sub(pattern1, '', text) 

    # Regular expression pattern to match URLs
    regex1 = '(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-&?=%.]+'
    regex2 = '(http|ftp|https|www)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?'

    regex_for_email = '[\w.+-]+@[\w-]+\.[\w.-]+'
    email_pattern = re.compile(regex_for_email)
    
    # Replace emails with <EMAIL> placeholder
    text = re.sub(email_pattern,'<EMAIL>',text)

    #  working best
    regex_for_website = '([\w+]+\:\/\/)?([\w\d-]+\.)*[\w-]+[\.\:]\w+([\/\?\=\&\#.]?[\w-]+)*\/?'
    url_pattern = re.compile(regex_for_website)

    # Replace URLs with <URL> placeholder
    text = re.sub(url_pattern, "<URL>", text)

    # Regular expression pattern to match hashtags
    hashtag_pattern = re.compile(r'#\w+')

    # Replace hashtags with <HASHTAG> placeholder
    text = re.sub(hashtag_pattern, '<HASHTAG>', text)

    # Regular expression pattern to match mentions
    mention_pattern = re.compile(r'@\w+')

    # Replace mentions with <MENTION> placeholder
    text = re.sub(mention_pattern, '<MENTION>', text)

    # removing unwanted special characters (not punctuations)
    # text = re.sub(r'[*|^|~|\/|+|_|$]',r' ',text)
    text = re.sub(r'(\W)\1+', r'\1', text)
    # text = re.sub(r'[^a-zA-Z0-9<>\n]',r'\s\1\s',text)
    # print(text)
    # Tokenize text into words and punctuation
    # r'\b\w+\b|[^\w\s]+'
    pattern = r'(?:[\w]+|<HASHTAG>|<MENTION>|<URL>|<NUMBER>|<EMAIL>|<s>|<\s>|[.?!,;:_\(\)\[\]/\-\"\']+)'
    # words = re.findall(r'\b\w+\b|[^\w\s]+', text)
    words = re.findall(pattern,text)
    return words






# N gram Generation

In [190]:

def get_ngrams(tokens):
  unigram,bigram,trigram,fourgram=dict(),dict(),dict(),dict()
  size = len(tokens)
  for i in range(size):
    
    unigram[tokens[i]] = unigram.get(tokens[i],0)+1
    
    if i+1<size:
        if bigram.get(tokens[i],0) == 0:
          bigram[tokens[i]] = dict()
        bigram[tokens[i]][tokens[i+1]] = bigram[tokens[i]].get(tokens[i+1],0)+1
        
    if i+2<size:
        if trigram.get(tokens[i]+' '+tokens[i+1],0) == 0:
            trigram[tokens[i]+' '+tokens[i+1]] = dict()
        trigram[tokens[i]+' '+tokens[i+1]][tokens[i+2]] = trigram[tokens[i]+' '+tokens[i+1]].get(tokens[i+2],0) + 1

    if i+3<size:
        if fourgram.get(tokens[i]+' '+tokens[i+1]+' '+tokens[i+2],0) == 0:
            fourgram[tokens[i]+' '+tokens[i+1]+' '+tokens[i+2]] = dict()
        fourgram[tokens[i]+' '+tokens[i+1]+' '+tokens[i+2]][tokens[i+3]] = fourgram[tokens[i]+' '+tokens[i+1]+' '+tokens[i+2]].get(tokens[i+3],0) + 1
  
  return unigram, bigram, trigram, fourgram
        

# Witten Bell Smoothing

In [191]:
def calc_unigram_wb_probability(unigram,unigram_header):
  res = 0
  den = 0
  lambda1 = 0
  unigram_word_count = unigram.get(unigram_header,0)

  total_unigrams = 0
  for key,value in unigram.items():
    total_unigrams+=value
  lambda1 = 1 - (len(unigram)/(len(unigram)+total_unigrams))
  res = ((lambda1 * unigram_word_count) / total_unigrams) + (1 - lambda1) / len(unigram)

  return res


In [192]:
def calc_bigram_wb_probability(unigram, bigram, bigram_context,bigram_header):
  res = 0
  den = 0
  lambda1 = 0
  first = 0
  if bigram.get(bigram_context) == 0:
    for key,value in bigram[bigram_context].items():
      den +=value
    num = bigram[bigram_context].get(bigram_header,0)

    if den == 0:
      den = 1 
    
    first = num / den

    given_bigram_types = len(bigram[bigram_context])
    given_bigram_T = den

    #  1 - lambda1 = N / ( N + T )
    lambda1 = 1 - ( (given_bigram_types)/(given_bigram_types + given_bigram_T) )
  
  # unigram_context = trigram_context.split()[-1]
  unigram_header = bigram_header
  res = lambda1 * first + (1-lambda1) * (calc_unigram_wb_probability(unigram,unigram_header))
  return res

In [193]:
def calc_trigram_wb_probability(unigram,bigram,trigram,trigram_context,trigram_header):
  res=0
  den=0
  lambda1=0
  first = 0
  # print(trigram_context)
  if trigram.get(trigram_context,0) != 0: 
    for key,value in trigram[trigram_context].items():
      den +=value
    num = trigram[trigram_context].get(trigram_header,0)

    if den == 0:
      den = 1
    first = num / den

    given_trigram_types = len(trigram[trigram_context])
    given_trigram_T = den

    #  1 - lambda1 = N / ( N + T )
    lambda1 = 1 - ( (given_trigram_types)/(given_trigram_types + given_trigram_T) )
  
  bigram_context = ' '.join(trigram_context.split()[-1:])
  bigram_header = trigram_header
  res = lambda1 * first + (1-lambda1) * (calc_bigram_wb_probability(unigram,bigram,bigram_context,bigram_header))
  return res

In [194]:
def calc_fourgram_wb_probability(unigram,bigram,trigram,fourgram,fourgram_context,fourgram_header):
  res=0
  den=0
  lambda1 = 0
  first = 0
  if fourgram.get(fourgram_context,0) != 0:
    for key,value in fourgram[fourgram_context].items():
      den +=value
    num = fourgram[fourgram_context].get(fourgram_header,0)

    if den == 0:
      den = 1
    first = num / den

    given_fourgram_types = len(fourgram[fourgram_context])
    given_fourgram_T = den

    #  1 - lambda1 = N / ( N + T )
    lambda1 = 1 - ( (given_fourgram_types)/(given_fourgram_types + given_fourgram_T) )

  trigram_context = ' '.join(fourgram_context.split()[-2:])
  trigram_header = fourgram_header
  res = lambda1 * first + (1-lambda1) * (calc_trigram_wb_probability(unigram,bigram,trigram,trigram_context,trigram_header))
  return res


# K N Smoothing

In [195]:
def calc_unigram_kn_prob(unigram,bigram,trigram,fourgram,d,fourgram_context,fourgram_header):
  num = 0
  den = 0
  first = 0
  lambda1 = 0
  N = 0
  T = 0

  unigram_context = fourgram_context.split()[-1]
  for key, header_dict in bigram.items():
    for header, val in header_dict.items():
      if header == unigram_context:
        num+=val
    den += len(header_dict)
  
  if den == 0:
    den = 1
  for key,val in unigram.items():
    T += val
  if T == 0:
    T = 1
  N = len(unigram)
  lambda1 = (d * N) / T
  if lambda1 == 0:
    lambda1 = 1
  res = max(num-d,0)/den + (lambda1 /len(unigram))
  return res


In [196]:
def calc_bigram_kn_prob(unigram,bigram,trigram,fourgram,d,fourgram_context,fourgram_header):
  num = 0
  den = 0
  first = 0
  lambda1 = 0
  N = 0
  T = 0
  # finding *cd
  bigram_context = fourgram_context.split()[-2]
  bigram_header = fourgram_context.split()[-1]

  # finding **c
  bigram_header2 = fourgram_context.split()[-2]
  for key, header_dict in trigram.items():
    last_1_part = key.split()[-1]
    for header, val in header_dict.items():
      if last_1_part == bigram_context and header == bigram_header:
        num += val
      if header == bigram_header2:
        den += val
  
  if den == 0:
    den = 1
  # [c] -> cd, ce, cf
  bigram_context2 = fourgram_context.split()[-1]
  if bigram.get(bigram_context2,0):
    for key,val in bigram[bigram_context2].items():
      T += val
    N = len(bigram[bigram_context2])
  if T == 0: 
    T = 1
  lambda1 = (d * N) / T
  if lambda1 == 0:
    lambda1 = 1
  res = max(num-d,0)/den + lambda1 * calc_unigram_kn_prob(unigram,bigram,trigram,fourgram,d,fourgram_context,fourgram_header)
  return res

In [197]:
def calc_trigram_kn_prob(unigram,bigram,trigram,fourgram,d,fourgram_context,fourgram_header):
  num=0
  den=0
  first = 0
  lambda1 = 0
  N = 0
  T = 0
  # finding *bcd
  trigram_context = " ".join(fourgram_context.split()[-2:])
  trigram_header = fourgram_header
 
  #finding **bc
  trigram_context2 = fourgram_context.split()[-2]
  trigram_header2 = fourgram_context.split()[-1]
  
  for key, header_dict in fourgram.items():
      last_2_parts = " ".join(key.split()[-2:])
      if last_2_parts == trigram_context:
        for header, val in header_dict.items():
          if header == trigram_header:
            num += val
      last_1_part = key.split()[-1]
      if last_1_part == trigram_context2:
        for header, val in header_dict.items():
          if header == trigram_header2:
            den+=val
  if den == 0:
    den = 1
  
  # [bc]->bcd, bce, bcf
  trigram_context3 = " ".join(fourgram_context.split()[-2:])
  if(trigram.get(trigram_context3,0)):
    for key, val in trigram[trigram_context3].items():
      T += val
    N = len(trigram[trigram_context3])

  if T == 0:
    T = 1
  lambda1 = (d * N) / T
  if lambda1 == 0:
    lambda1 = 1

  res = max(num-d,0)/den + lambda1 * calc_bigram_kn_prob(unigram,bigram,trigram,fourgram,d,fourgram_context,fourgram_header)
  return res
      

In [198]:
def calc_fourgram_kn_prob(unigram, bigram, trigram, fourgram,d,fourgram_context,fourgram_header):
  num=0
  den=0
  first = 0
  lambda1 = 0
  N = 0
  T = 0
  if fourgram.get(fourgram_context,0):
    num = fourgram[fourgram_context].get(fourgram_header,0)
    for key, val in fourgram[fourgram_context].items():
      den+=val
    N = len(fourgram[fourgram_context])
    
  if den == 0:
    den = 1 
  T = den
  lambda1 = (d*N)/T
  first = max(num-d,0) / den 
  if lambda1 == 0:
    lambda1 = 1 
  # print('4: ',first,lambda1)
  res = first + lambda1 * calc_trigram_kn_prob(unigram,bigram,trigram,fourgram,d,fourgram_context,fourgram_header)
  return res

# Main

In [200]:
def get_ngrams2(corpus,n):
    token = tokenize(corpus)
    ngrams = zip(*[token[i:] for i in range(n)])
    ngram_list = [" ".join(gram) for gram in ngrams]
    return ngram_list,len(token)

In [202]:
import numpy as np
def splitdata(sentence_list,n):
  test_indices = np.random.choice(len(sentence_list), size=n, replace=False)
  mask = np.ones(len(sentence_list), bool)
  mask[test_indices] = False
  train_set = list(np.array(sentence_list)[mask])
  test_set = list(np.array(sentence_list)[~mask])
  return train_set, test_set

In [203]:
def calc_perplexity_kn(unigram,bigram,trigram,fourgram,d,sentence):
  sentence = '<s>'+' '+'<s>'+' '+'<s>'+' '+sentence+' '+'<\s>'+' '+'<\s>'+' '+'<\s>'
  input_fourgram,n = get_ngrams2(sentence,4)
  res=1
  for gram in input_fourgram:
    context = " ".join(gram.split()[:-1])
    header = gram.split()[-1]
    res*=calc_fourgram_kn_prob(unigram,bigram,trigram,fourgram,d,context,header)
  res2 = pow(1/res,1/n)
  return res2

In [204]:
def calc_perplexity(unigram,bigram,trigram,fourgram,sentence):
  sentence = '<s>'+' '+'<s>'+' '+'<s>'+' '+sentence+' '+'<\s>'+' '+'<\s>'+' '+'<\s>'
  input_fourgram,n = get_ngrams2(sentence,4)
  res=1
  for gram in input_fourgram:
    context = " ".join(gram.split()[:-1])
    header = gram.split()[-1]
    res*=calc_fourgram_wb_probability(unigram,bigram,trigram,fourgram,context,header)
  res2 = pow(1/res,1/n)
  return res2
  



In [205]:
if __name__=="__main__":
    filename = '/content/drive/MyDrive/IIIT Hyderabad/Semester 4/Intro. To NLP/Assignment 1/hello.txt'
    filename = '/content/drive/MyDrive/IIIT Hyderabad/Semester 4/Intro. To NLP/Assignment 1/Pride and Prejudice - Jane Austen.txt'
    
    with open(filename,'r') as f:

        string = f.read()
        sentencelist=string.split('\n')
        train_set,test_set = splitdata(sentencelist,1000)
        tokens=[]
        for string in train_set:
          tokens.extend(tokenize(string))
        unigram, bigram, trigram, fourgram = get_ngrams(tokens)
        fourgram_context = 'maneesh is a'
        fourgram_header = 'girl'
        
        d=0.75
        print(calc_fourgram_wb_probability(unigram,bigram,trigram,fourgram,fourgram_context,fourgram_header))
        print(calc_fourgram_kn_prob(unigram, bigram, trigram, fourgram,d,fourgram_context,fourgram_header) )
        
        perplexity = 0
        for sentence in test_set:
            perplexity += calc_perplexity(unigram,bigram,trigram,fourgram,sentence)
            # print(perplexity)
        print("WB: ",perplexity/1000)

        perplexity2 = 0
        N = 10
        for sentence in test_set[:N]:
            perplexity2 += calc_perplexity_kn(unigram,bigram,trigram,fourgram,0.75,sentence)
        print("KN: ",perplexity2/N)
      

0.00010962837356947829
0.047579554261088314
WB:  119.00257716534502
KN:  73.72396911231135


# Useless

In [206]:
string = "My name is Maneesh Gupta"
trigram_context = ' '.join(string.split()[-3:])
print(trigram_context)

is Maneesh Gupta


In [207]:
l1 = zip(*[[1,2,3],[2,3,4],[3,4,5]])
print(*l1)

(1, 2, 3) (2, 3, 4) (3, 4, 5)



```
is a
1.2834039189034834e-05
```

